In [1]:
# Determine hot places in Singapore

In [124]:
# Import packages
import requests
import json
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import time

In [174]:
class singaporeAirTemperature:
    
    def __init__(self, startDate, numDays):
        presentDate = datetime.today()
        presentDateString = presentDate.strftime("%Y-%m-%d")
        try:
            startDateType = datetime.strptime(startDate, "%Y-%m-%d")
        except ValueError:
            return print("Invalid date format. Please follow format YYYY-MM-DD.")
        numDaysMax = (presentDate - startDateType).days
        self.startDate = startDate
        self.startDateType = startDateType
        if numDaysMax > 1:
            dateList = [numDays, numDaysMax]
            numDaysIndex = np.argmin(dateList, axis = 0) 
            numDays = dateList[numDaysIndex]
            self.numDays = numDays
            self.dates = [self.startDateType + timedelta(days = i) for i in range(0, (self.numDays))]
        else:
            raise Exception(f"Number of days between {self.startDate} and {presentDateString} is less than 1 day")
            
    def getData(self):
        url = "https://api.data.gov.sg/v1/environment/air-temperature"
        headers = {"accept": "application/json"}
        dataJsonList = []
        counter = 0
        print("Begin data loading...")
        for i in [j.strftime("%Y-%m-%d") for j in self.dates]:
            response = requests.request("GET", url, headers = headers, params = params)
            time.sleep(3)
            data = response.text
            dataJson = json.loads(data)
            dataJsonList.append(dataJson)
            counter += 1
            numDaysRemaining = self.numDays - counter
            print(f"Data loaded for date {i} with {numDaysRemaining} days remaining...")
            if counter == self.numDays:
                print("Data loading completed.")  
        return dataJsonList
    
    def loadData(self):
        dataJsonList = self.getData()
        dfReadings = pd.DataFrame()
        for i in dataJsonList[0]["items"]:
            timestamp = i["timestamp"]
            dfR = pd.DataFrame(i["readings"])
            dfR["timestamp"] = i["timestamp"]
            dfReadings = pd.concat([dfReadings, dfR], axis = 0)
            dfReadings = dfReadings.reset_index(drop = True)
        dfStations = pd.DataFrame()
        for j in dataJsonList:
            dfS = pd.DataFrame(j["metadata"]["stations"])
            dfStations = pd.concat([dfStations, dfS], axis = 0)
            dfStations = dfStations.reset_index(drop = True)
        print(f"Size of dfReadings: {dfReadings.shape}\nSize of dfStations: {dfStations.shape}")
        return dfReadings, dfStations

In [175]:
temp = singaporeAirTemperature("2021-10-01", 10)

In [176]:
dfReadings, dfStations = temp.loadData()

Begin data loading...
Data loaded for date 2021-10-01 with 9 days remaining...
Data loaded for date 2021-10-02 with 8 days remaining...
Data loaded for date 2021-10-03 with 7 days remaining...
Data loaded for date 2021-10-04 with 6 days remaining...
Data loaded for date 2021-10-05 with 5 days remaining...
Data loaded for date 2021-10-06 with 4 days remaining...
Data loaded for date 2021-10-07 with 3 days remaining...
Data loaded for date 2021-10-08 with 2 days remaining...
Data loaded for date 2021-10-09 with 1 days remaining...
Data loaded for date 2021-10-10 with 0 days remaining...
Data loading completed.
Size of dfReadings: (20053, 3)
Size of dfStations: (140, 4)
